# ParallelGNN trf learning model tuning
1. This notebook will be focused on hyperparameter tuning for the trf learning model.
2. Hyperparameters were tuned using solubility dataset which is in the data/graph_data/logS_train and logS_val folders
3. To prevent any sort of test leakage, molecules that appear in both solubililty and oral bioavailability dataset have been removed from the solubililty dataset prior to tuning

## Note:
Suggest to skip this notebook and refer to the best parameters which were obtained and saved in config.py as best_params_parallel

In [ ]:
# import all required materials

import torch
import numpy as np
from torch_geometric.loader import DataLoader
import optuna

from model import ParallelGNN
from config import NUM_FEATURES, NUM_TARGET, EDGE_DIM, DEVICE, SEED_NO, PATIENCE, EPOCHS, NUM_GRAPHS_PER_BATCH
from engine import EngineSol
from utils import seed_everything, LoadSolDataset

In [6]:
seed_everything(SEED_NO)

# load in data
train_dataset = LoadSolDataset(root='./data/graph_data/logS_train/', raw_filename='trainset_removed.csv')
val_dataset = LoadSolDataset(root='./data/graph_data/logS_val/', raw_filename='valset_removed.csv')

print(len(train_dataset))
print(len(val_dataset))

7880
978


In [7]:
def run_tuning(train_dataset, valid_dataset, params):
    seed_everything(SEED_NO)
    print(len(train_dataset))
    print(len(valid_dataset))
    
    train_loader = DataLoader(train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)
    model = ParallelGNN(num_features=NUM_FEATURES, num_targets=NUM_TARGET, num_gin_layers=params['num_gin_layers'], num_graph_trans_layers=params['num_graph_trans_layers'], 
                            hidden_size=params['hidden_size'], n_heads=params['n_heads'], dropout=params['dropout'], edge_dim=EDGE_DIM)
    model.to(DEVICE)
    optimizer=torch.optim.Adam(model.parameters(),lr = params['learning_rate'])
    eng = EngineSol(model, optimizer, device=DEVICE)

    best_loss = np.inf
    early_stopping_iter = PATIENCE
    early_stopping_counter = 0 

    for epoch in range(EPOCHS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.validate(valid_loader)
        print(f'Epoch: {epoch+1}/{EPOCHS}, train loss : {train_loss}, validation loss : {valid_loss}')
        if valid_loss < best_loss:
            best_loss = valid_loss 
            early_stopping_counter=0

        else:
            early_stopping_counter +=1

        if early_stopping_counter > early_stopping_iter:
            print('Early stopping...')
            break
        print(f'Early stop counter: {early_stopping_counter}')
    
    return best_loss

In [8]:
def objective(trial):
    params = {
        'num_gin_layers' : trial.suggest_int('num_gin_layers', 1,3),
        'num_graph_trans_layers' : trial.suggest_int('num_graph_trans_layers', 1,3),
        'hidden_size' : trial.suggest_int('hidden_size', 64, 512),
        'n_heads' : trial.suggest_int('n_heads', 1, 5),
        'dropout': trial.suggest_float('dropout', 0.1,0.4),
        'learning_rate' : trial.suggest_float('learning_rate', 1e-3, 9e-3, log=True),
    }
    
    loss = run_tuning(train_dataset, val_dataset, params)
    return loss

if __name__ == '__main__':
    study = optuna.create_study(direction = 'minimize')
    study.optimize(objective, n_trials=30)
    print(f'best trial:')
    trial_ = study.best_trial
    print(trial_.values)
    best_parameters= trial_.params
    print(f'Best parameters: {trial_.params}')

[I 2023-03-10 13:56:09,788] A new study created in memory with name: no-name-6231db9f-8a60-458a-b2f8-7a33756bc29f


7880
978
Epoch: 1/300, train loss : 290.3271399851768, validation loss : 2.22818723320961
Early stop counter: 0
Epoch: 2/300, train loss : 2.606668972199963, validation loss : 1.9741106927394867
Early stop counter: 0
Epoch: 3/300, train loss : 1.9449950187436995, validation loss : 1.5947377681732178
Early stop counter: 0
Epoch: 4/300, train loss : 1.6714182938298872, validation loss : 1.3680839240550995
Early stop counter: 0
Epoch: 5/300, train loss : 1.5444592852746286, validation loss : 1.2616454362869263
Early stop counter: 0
Epoch: 6/300, train loss : 1.5998286585653982, validation loss : 1.2205303013324738
Early stop counter: 0
Epoch: 7/300, train loss : 1.519472018364937, validation loss : 1.2230390310287476
Early stop counter: 1
Epoch: 8/300, train loss : 1.2800405294664445, validation loss : 1.0762037932872772
Early stop counter: 0
Epoch: 9/300, train loss : 1.3477996299343724, validation loss : 1.167194128036499
Early stop counter: 1
Epoch: 10/300, train loss : 1.1653361416632

[I 2023-03-10 13:58:10,775] Trial 0 finished with value: 0.7135431319475174 and parameters: {'num_gin_layers': 2, 'num_graph_trans_layers': 2, 'hidden_size': 357, 'n_heads': 4, 'dropout': 0.28120954888547883, 'learning_rate': 0.004671688214963837}. Best is trial 0 with value: 0.7135431319475174.


Epoch: 56/300, train loss : 0.7955415710326164, validation loss : 0.7708579301834106
Early stopping...
7880
978
Epoch: 1/300, train loss : 5.043731435652702, validation loss : 2.120346248149872
Early stop counter: 0
Epoch: 2/300, train loss : 2.220684532196291, validation loss : 1.8815438151359558
Early stop counter: 0
Epoch: 3/300, train loss : 1.7654279431989115, validation loss : 1.4405556321144104
Early stop counter: 0
Epoch: 4/300, train loss : 1.5212061559000323, validation loss : 1.3542621731758118
Early stop counter: 0
Epoch: 5/300, train loss : 1.4453640599404611, validation loss : 1.0889069139957428
Early stop counter: 0
Epoch: 6/300, train loss : 1.2984977960586548, validation loss : 1.0574576258659363
Early stop counter: 0
Epoch: 7/300, train loss : 1.127711409522641, validation loss : 0.9298200756311417
Early stop counter: 0
Epoch: 8/300, train loss : 1.073629585004622, validation loss : 0.8416131287813187
Early stop counter: 0
Epoch: 9/300, train loss : 1.058524973930851,

[I 2023-03-10 14:03:16,874] Trial 1 finished with value: 0.5335602015256882 and parameters: {'num_gin_layers': 2, 'num_graph_trans_layers': 2, 'hidden_size': 394, 'n_heads': 5, 'dropout': 0.34959517368819026, 'learning_rate': 0.0011455519465425172}. Best is trial 1 with value: 0.5335602015256882.


...

Epoch: 54/300, train loss : 0.46868155175639736, validation loss : 0.697597861289978
Early stopping...
7880
978
Epoch: 1/300, train loss : 6.607917947153891, validation loss : 2.2860641479492188
Early stop counter: 0
Epoch: 2/300, train loss : 2.073849058920337, validation loss : 1.6139551401138306
Early stop counter: 0
Epoch: 3/300, train loss : 1.59355289705338, validation loss : 1.158608227968216
Early stop counter: 0
Epoch: 4/300, train loss : 1.4087601323281564, validation loss : 1.000592976808548
Early stop counter: 0
Epoch: 5/300, train loss : 1.2726705324265264, validation loss : 0.9348074793815613
Early stop counter: 0
Epoch: 6/300, train loss : 1.1718698701550883, validation loss : 0.8789867609739304
Early stop counter: 0
Epoch: 7/300, train loss : 1.0614297082347255, validation loss : 0.8598837852478027
Early stop counter: 0
Epoch: 8/300, train loss : 1.06193813777739, validation loss : 0.9711734652519226
Early stop counter: 1
Epoch: 9/300, train loss : 1.0403018459197013, v

[I 2023-03-10 14:38:39,571] Trial 28 finished with value: 0.5756497830152512 and parameters: {'num_gin_layers': 3, 'num_graph_trans_layers': 2, 'hidden_size': 481, 'n_heads': 2, 'dropout': 0.1744188717004227, 'learning_rate': 0.0014002147306439784}. Best is trial 19 with value: 0.5163337364792824.


Epoch: 53/300, train loss : 0.6277645557157455, validation loss : 0.657489538192749
Early stopping...
7880
978
Epoch: 1/300, train loss : 12.891149059418709, validation loss : 2.4169867634773254
Early stop counter: 0
Epoch: 2/300, train loss : 2.024635761014877, validation loss : 1.4513471126556396
Early stop counter: 0
Epoch: 3/300, train loss : 1.5413436235920075, validation loss : 1.5352289080619812
Early stop counter: 1
Epoch: 4/300, train loss : 1.3693953291062386, validation loss : 1.1090620458126068
Early stop counter: 0
Epoch: 5/300, train loss : 1.2437981847793824, validation loss : 1.4412329196929932
Early stop counter: 1
Epoch: 6/300, train loss : 1.2285870082916752, validation loss : 0.9050513505935669
Early stop counter: 0
Epoch: 7/300, train loss : 1.1823838718475834, validation loss : 0.9618768393993378
Early stop counter: 1
Epoch: 8/300, train loss : 1.1016840281025055, validation loss : 0.8990818560123444
Early stop counter: 0
Epoch: 9/300, train loss : 1.0214762245455

[I 2023-03-10 14:39:57,976] Trial 29 finished with value: 0.5779952183365822 and parameters: {'num_gin_layers': 2, 'num_graph_trans_layers': 2, 'hidden_size': 365, 'n_heads': 1, 'dropout': 0.28627296075369973, 'learning_rate': 0.002284665060321953}. Best is trial 19 with value: 0.5163337364792824.


Epoch: 71/300, train loss : 0.5071798293821274, validation loss : 0.5799025297164917
Early stopping...
best trial:
[0.5163337364792824]
Best parameters: {'num_gin_layers': 2, 'num_graph_trans_layers': 2, 'hidden_size': 396, 'n_heads': 1, 'dropout': 0.21481760818350845, 'learning_rate': 0.0012104847904387125}


Hyperparameters were saved in config.py as best_params_parallel